In [1]:
import glob
import random
from typing import List, Tuple
import numpy as np

import tensorflow as tf

import json

In [2]:
import keras
from keras.layers import Dense, LSTM, SeparableConv1D, Flatten, Conv1D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import time

In [3]:
Segment = str
Genome = List[Segment]

Coronaviridae = glob.glob("./dataset10/Coronaviridae/*.fna")
Ebolavirus = glob.glob("./dataset10/Ebolavirus/*.fna")
HepatitisB = glob.glob("./dataset10/HepatitisB/*.fna")
Herpes = glob.glob("./dataset10/Herpes/*.fna")
HIV = glob.glob("./dataset10/HIV/*.fna")
InfluenzaA = glob.glob("./dataset10/InfluenzaA/*.fna")
Metapneumovirus = glob.glob("./dataset10/Metapneumovirus/*.fna")
Papillomaviridae = glob.glob("./dataset10/Papillomaviridae/*.fna")
Rhinovirus = glob.glob("./dataset10/Rhinovirus/*.fna")
SARS_CoV_2 = glob.glob("./dataset10/SARS-CoV-2/*.fna")

print(Coronaviridae)

['./dataset10/Coronaviridae\\GCF_000848685.1_ViralProj14739_genomic.fna', './dataset10/Coronaviridae\\GCF_000853505.1_ViralProj14913_genomic.fna', './dataset10/Coronaviridae\\GCF_000853865.1_ViralProj14960_genomic.fna', './dataset10/Coronaviridae\\GCF_000856025.1_ViralProj15097_genomic.fna', './dataset10/Coronaviridae\\GCF_000858765.1_ViralProj15139_genomic.fna', './dataset10/Coronaviridae\\GCF_000862345.1_ViralProj15350_genomic.fna', './dataset10/Coronaviridae\\GCF_000862505.1_ViralProj15385_genomic.fna', './dataset10/Coronaviridae\\GCF_000862965.1_ViralProj15303_genomic.fna', './dataset10/Coronaviridae\\GCF_000864885.1_ViralProj15500_genomic.fna', './dataset10/Coronaviridae\\GCF_000868045.1_ViralProj18867_genomic.fna', './dataset10/Coronaviridae\\GCF_000868165.1_ViralProj17048_genomic.fna', './dataset10/Coronaviridae\\GCF_000870505.1_ViralProj18863_genomic.fna', './dataset10/Coronaviridae\\GCF_000870985.1_ViralProj20135_genomic.fna', './dataset10/Coronaviridae\\GCF_000872845.1_ViralP

In [4]:
def getTrainAndValid(viruses_list: List[List[str]]) -> List[Tuple[List[str], List[str]]]:
    rand_device = random.Random(0)
    split_list = []
    for virus in viruses_list:
        files_list = virus.copy()
        rand_device.shuffle(files_list)
        split_idx = int(0.8*len(files_list))
        split_list.append((files_list[:split_idx], files_list[split_idx:]))
    return split_list

In [5]:
def buildGenome(filename: str) -> str:
    genome_file = open(filename, 'r')
    genome: Genome = []
    genome_idx = -1
    for line in genome_file:
        if line[0] == ">":
            genome.append("")
            genome_idx += 1
            continue
        else:
            genome[genome_idx] += line[:-1]
    genome_file.close
    return genome

train_and_valid_files = getTrainAndValid([Coronaviridae, Ebolavirus, HepatitisB, Herpes, HIV, InfluenzaA, Metapneumovirus, Papillomaviridae, Rhinovirus, SARS_CoV_2])
genome = buildGenome(train_and_valid_files[0][0][0])
print(genome)

['...']


In [6]:
def getSample(genome: Genome):
    fragment_len = 150
    samples = []
    for segment in genome:
        num_of_frags_in_seg = len(segment) - fragment_len + 1
        sample_prob = 0.05
        should_sample = np.random.uniform(size=num_of_frags_in_seg) < sample_prob
        for i in range(num_of_frags_in_seg):
            if should_sample[i]:
                samples.append(segment[i:i+fragment_len])
    return samples
                
print(len(getSample(genome)[6]))

150


In [7]:
A_tensor = np.array([1, 0, 0, 0])
C_tensor = np.array([0, 1, 0, 0])
G_tensor = np.array([0, 0, 1, 0])
T_tensor = np.array([0, 0, 0, 1])
N_tensor = np.array([0, 0, 0, 0])

Example = np.ndarray
Label = np.ndarray

# def label_tensor(str_label):
#     if str_label == 'Coronaviridae':
#         label = np.array([1, 0, 0, 0, 0, 0])
#     elif str_label == 'Ebolavirus':
#         label = np.array([0, 1, 0, 0, 0, 0])
#     elif str_label == 'HepatitisB':
#         label = np.array([0, 0, 1, 0, 0, 0])
#     elif str_label == 'Herpes':
#         label = np.array([0, 0, 0, 1, 0, 0])
#     elif str_label == 'HIV':
#         label = np.array([0, 0, 0, 0, 1, 0])
#     elif str_label == 'InfluenzaA':
#         label = np.array([0, 0, 0, 0, 0, 1])
#     elif str_label == 'Metapneumovirus':
#         label = np.array([0, 1, 0, 0, 0, 0])
#     elif str_label == 'Papillomaviridae':
#         label = np.array([0, 0, 1, 0, 0, 0])
#     elif str_label == 'Rhinovirus':
#         label = np.array([0, 0, 0, 1, 0, 0])
#     elif str_label == 'SARS_CoV_2':
#         label = np.array([0, 0, 0, 0, 1, 0])
#     return label

# def tensorize(samples: List[str], label: np.array) -> List[Tuple[Example, Label]]:
#     tensors = []
#     for sample in samples:
#         # Build the appropriate tensor
#         tensor = np.zeros(shape=(len(sample), 4))
#         for i in range(len(sample)):
#             base = sample[i]
#             if base == 'A':
#                 tensor[i][:] = A_tensor
#             elif base == 'C':
#                 tensor[i][:] = C_tensor
#             elif base == 'G':
#                 tensor[i][:] = G_tensor
#             elif base == 'T':
#                 tensor[i][:] = T_tensor
#             else:
#                 tensor[i][:] = N_tensor
#         # Insert it to the tensors list
#         yield (tensor, label_tensor(str_label))

In [8]:
def generator(virus_dirs, is_train: bool):
    # We are going to find the viruses
    viruses_list = []
    for virus in virus_dirs:
        viruses_list.append(virus)
    onehot_encoding = {}
    raw_dataset = getTrainAndValid(viruses_list)
    if is_train:
        dataset_idx = 0
    else:
        dataset_idx = 1
    virus_idx = 0
    for virus in raw_dataset:
        label = np.zeros(len(raw_dataset))
        label[virus_idx] = 1
        for genome_file in virus[dataset_idx]:
            genome = buildGenome(genome_file)
            samples = getSample(genome)
            # Tensorize       
            for sample in samples:
                #Build the appropriate tensor
                tensor = np.zeros(shape=(len(sample), 4))
                for i in range(len(sample)):
                    base = sample[i]
                    if base == 'A':
                        tensor[i][:] = A_tensor
                    elif base == 'C':
                        tensor[i][:] = C_tensor
                    elif base == 'G':
                        tensor[i][:] = G_tensor
                    elif base == 'T':
                        tensor[i][:] = T_tensor
                    else:
                        tensor[i][:] = N_tensor
                # Insert it to the tensors list
                yield (tensor, label)
        virus_idx += 1

In [38]:
base_count = 4
classes = [Coronaviridae, Ebolavirus, HepatitisB, Herpes, HIV, InfluenzaA, Metapneumovirus, Papillomaviridae, Rhinovirus, SARS_CoV_2]

trainset = tf.data.Dataset.from_generator(
    generator = lambda: generator(classes, True),
    output_signature=(
        tf.TensorSpec(shape=(150, base_count), dtype=tf.float32),
        tf.TensorSpec(shape=(len(classes)), dtype=tf.float32)
    )
)

trainset = trainset.batch(64).repeat()

In [49]:
validset = tf.data.Dataset.from_generator(
    generator = lambda: generator(classes, False).repeat(),
    output_signature=(
        tf.TensorSpec(shape=(150, base_count), dtype=tf.float32),
        tf.TensorSpec(shape=(len(classes)), dtype=tf.float32)
    )
)

validset = validset.batch(64)

In [ ]:
amount_trainset=0
for examples in trainset:
    amount_trainset=amount_trainset+1
print(amount_trainset)

In [43]:
checkpointSeq = ModelCheckpoint('modelSeq.hdf5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [11]:
checkpointFF = ModelCheckpoint('modelFF.hdf5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [ ]:
checkpointRNN = ModelCheckpoint('modelRNN.hdf5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [ ]:
checkpointCNN = ModelCheckpoint('modelCNN.hdf5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [11]:
checkpointCNNsep = ModelCheckpoint('modelCNNsep.hdf5', monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [40]:
'''
Our input is a matrix of size 150x4
1. we should trasform it into 600 element vector
2. pass is through an MLP layer with 10 neurons
'''
modelSeq = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[150, 4]),
    tf.keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal")
])

In [41]:
modelSeq.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 600)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                6010      
                                                                 
Total params: 6,010
Trainable params: 6,010
Non-trainable params: 0
_________________________________________________________________


In [42]:
modelSeq.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

In [45]:
time0 = time.time()

historySeq = modelSeq.fit(trainset, steps_per_epoch=int(842749/64), epochs = 20, callbacks = [checkpointSeq], verbose = 1)

print('Time taken to run 20 epochs: ', time.time()-time0)

Epoch 1/20
13167/13167 [==============================] - 246s 19ms/step - loss: 0.0350 - accuracy: 0.9932
Epoch 2/20
13167/13167 [==============================] - 263s 20ms/step - loss: 0.0315 - accuracy: 0.9937
Epoch 3/20
13167/13167 [==============================] - 262s 20ms/step - loss: 0.0305 - accuracy: 0.9937
Epoch 4/20
13167/13167 [==============================] - 267s 20ms/step - loss: 0.0298 - accuracy: 0.9937
Epoch 5/20
13167/13167 [==============================] - 275s 21ms/step - loss: 0.0292 - accuracy: 0.9938
Epoch 6/20
13167/13167 [==============================] - 273s 21ms/step - loss: 0.0291 - accuracy: 0.9936
Epoch 7/20
13167/13167 [==============================] - 276s 21ms/step - loss: 0.0291 - accuracy: 0.9937
Epoch 8/20
13167/13167 [==============================] - 274s 21ms/step - loss: 0.0292 - accuracy: 0.9936
Epoch 9/20
13167/13167 [==============================] - 268s 20ms/step - loss: 0.0292 - accuracy: 0.9936
Epoch 10/20
13167/13167 [============

In [49]:
modelFF = Sequential()
modelFF.add(Flatten(input_shape=[150, 4]))
modelFF.add(Dense(256,  activation = 'relu'))
modelFF.add(Dense(128, activation = 'relu'))
modelFF.add(Dense(128, activation = 'relu'))
modelFF.add(Dense(128, activation = 'relu'))
modelFF.add(Dense(10, activation = 'softmax'))

modelFF.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']);

In [50]:
modelFF.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 600)               0         
                                                                 
 dense_20 (Dense)            (None, 256)               153856    
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dense_23 (Dense)            (None, 128)               16512     
                                                                 
 dense_24 (Dense)            (None, 10)                1290      
                                                                 
Total params: 221,066
Trainable params: 221,066
Non-tr

In [52]:
time0 = time.time()

historyFF = modelFF.fit(trainset, steps_per_epoch=int(842749/64), epochs = 20, callbacks = [checkpointFF], verbose = 1);

print('Time taken to run 100 epochs: ', time.time()-time0)

Epoch 1/20
13167/13167 [==============================] - 270s 21ms/step - loss: 0.1988 - accuracy: 0.9914
Epoch 2/20
13167/13167 [==============================] - 285s 22ms/step - loss: 0.0836 - accuracy: 0.9857
Epoch 3/20
13167/13167 [==============================] - 287s 22ms/step - loss: 0.0934 - accuracy: 0.9880
Epoch 4/20
13167/13167 [==============================] - 290s 22ms/step - loss: 0.0962 - accuracy: 0.9881
Epoch 5/20
13167/13167 [==============================] - 285s 22ms/step - loss: 0.0734 - accuracy: 0.9868
Epoch 6/20
13167/13167 [==============================] - 373s 28ms/step - loss: 0.0786 - accuracy: 0.9917
Epoch 7/20
13167/13167 [==============================] - 295s 22ms/step - loss: 0.0861 - accuracy: 0.9840
Epoch 8/20
13167/13167 [==============================] - 612s 46ms/step - loss: 0.1180 - accuracy: 0.9798
Epoch 9/20
13167/13167 [==============================] - 770s 58ms/step - loss: 0.1164 - accuracy: 0.9816
Epoch 10/20
13167/13167 [============

In [13]:
modelRNN = Sequential()
modelRNN.add(LSTM(3, input_shape = (150,4)))
modelRNN.add(Dense(10, activation = 'softmax'))

modelRNN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'], )

In [14]:
modelRNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3)                 96        
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________


In [16]:
time0 = time.time()

historyRNN = modelRNN.fit(trainset, steps_per_epoch=int(842749/64), epochs = 20, callbacks = [checkpointRNN], verbose = 1);

print('Time taken to run 20 epochs: ', time.time()-time0)

Epoch 1/20
13167/13167 [==============================] - 771s 58ms/step - loss: 0.2186 - accuracy: 0.9624
Epoch 2/20
13167/13167 [==============================] - 789s 60ms/step - loss: 0.1108 - accuracy: 0.9750
Epoch 3/20
13167/13167 [==============================] - 800s 61ms/step - loss: 0.0866 - accuracy: 0.9796
Epoch 4/20
13167/13167 [==============================] - 811s 62ms/step - loss: 0.1027 - accuracy: 0.9752
Epoch 5/20
13167/13167 [==============================] - 1268s 96ms/step - loss: 0.1102 - accuracy: 0.9721
Epoch 6/20
13167/13167 [==============================] - 1395s 106ms/step - loss: 0.0846 - accuracy: 0.9810
Epoch 7/20
13167/13167 [==============================] - 1397s 106ms/step - loss: 0.1085 - accuracy: 0.9753
Epoch 8/20
13167/13167 [==============================] - 1397s 106ms/step - loss: 0.1170 - accuracy: 0.9684
Epoch 9/20
13167/13167 [==============================] - 1402s 106ms/step - loss: 0.0749 - accuracy: 0.9851
Epoch 10/20
13167/13167 [===

In [20]:
modelCNN = Sequential()

modelCNN.add(Conv1D(128, 2, activation = 'relu', input_shape = [150,4]))
modelCNN.add(Conv1D(64, 3, activation = 'relu'))
modelCNN.add(Conv1D(64, 1, activation = 'relu'))
modelCNN.add(Conv1D(64, 1, activation = 'relu'))
modelCNN.add(Conv1D(64, 1, activation = 'relu'))
modelCNN.add(Flatten(input_shape = [150,4]))
modelCNN.add(Dense(10, activation = 'softmax'))

modelCNN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [21]:
modelCNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 149, 128)          1152      
                                                                 
 conv1d_1 (Conv1D)           (None, 147, 64)           24640     
                                                                 
 conv1d_2 (Conv1D)           (None, 147, 64)           4160      
                                                                 
 conv1d_3 (Conv1D)           (None, 147, 64)           4160      
                                                                 
 conv1d_4 (Conv1D)           (None, 147, 64)           4160      
                                                                 
 flatten (Flatten)           (None, 9408)              0         
                                                                 
 dense_1 (Dense)             (None, 10)               

In [24]:
time0 = time.time()

historyCNN = modelCNN.fit(trainset, steps_per_epoch=int(842749/64), epochs = 20, callbacks = [checkpointCNN], verbose = 1);

print('Time taken to run 20 epochs: ', time.time()-time0)

Epoch 1/20
13167/13167 [==============================] - 1941s 147ms/step - loss: 0.1958 - accuracy: 0.9671
Epoch 2/20
13167/13167 [==============================] - 1823s 138ms/step - loss: 0.9135 - accuracy: 0.7570
Epoch 3/20
13167/13167 [==============================] - 1072s 81ms/step - loss: 0.9071 - accuracy: 0.8129
Epoch 4/20
13167/13167 [==============================] - 1067s 81ms/step - loss: 0.9373 - accuracy: 0.8075
Epoch 5/20
13167/13167 [==============================] - 1077s 82ms/step - loss: 0.9282 - accuracy: 0.8086
Epoch 6/20
13167/13167 [==============================] - 1089s 83ms/step - loss: 0.9177 - accuracy: 0.8110
Epoch 7/20
13167/13167 [==============================] - 1044s 79ms/step - loss: 0.9201 - accuracy: 0.8095
Epoch 8/20
13167/13167 [==============================] - 1044s 79ms/step - loss: 0.9164 - accuracy: 0.8096
Epoch 9/20
13167/13167 [==============================] - 1052s 80ms/step - loss: 0.9197 - accuracy: 0.8100
Epoch 10/20
13167/13167 [=

In [12]:
modelCNNsep = Sequential()
modelCNNsep.add(SeparableConv1D(128, 2, activation = 'relu',  input_shape = (150,4)))
modelCNNsep.add(SeparableConv1D(64, 3, activation = 'relu'))
modelCNNsep.add(SeparableConv1D(64, 1, activation = 'relu'))
modelCNNsep.add(SeparableConv1D(64, 1, activation = 'relu'))
modelCNNsep.add(SeparableConv1D(64, 1, activation = 'relu'))
modelCNNsep.add(Flatten())
modelCNNsep.add(Dense(10, activation = 'softmax'))

# modelCNNsep = Sequential()
# modelCNNsep.add(SeparableConv1D(4, 2, activation = 'relu',  input_shape = (150,4)))
# modelCNNsep.add(SeparableConv1D(3, 3, activation = 'relu'))
# modelCNNsep.add(SeparableConv1D(2, 1, activation = 'relu'))
# modelCNNsep.add(SeparableConv1D(2, 1, activation = 'relu'))
# modelCNNsep.add(SeparableConv1D(2, 1, activation = 'relu'))
# modelCNNsep.add(Flatten())
# modelCNNsep.add(Dense(10, activation = 'softmax'))

modelCNNsep.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
modelCNNsep.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv1d (Separable  (None, 149, 128)         648       
 Conv1D)                                                         
                                                                 
 separable_conv1d_1 (Separab  (None, 147, 64)          8640      
 leConv1D)                                                       
                                                                 
 separable_conv1d_2 (Separab  (None, 147, 64)          4224      
 leConv1D)                                                       
                                                                 
 separable_conv1d_3 (Separab  (None, 147, 64)          4224      
 leConv1D)                                                       
                                                                 
 separable_conv1d_4 (Separab  (None, 147, 64)          4

In [14]:
time0 = time.time()

historyCNNsep = modelCNNsep.fit(trainset, steps_per_epoch=int(842749/64), epochs = 20, callbacks = [checkpointCNNsep], verbose = 1);

print('Time taken to run 20 epochs: ', time.time()-time0)

Epoch 1/20
13167/13167 [==============================] - 1729s 131ms/step - loss: 0.0953 - accuracy: 0.9917
Epoch 2/20
13167/13167 [==============================] - 1325s 101ms/step - loss: 0.1084 - accuracy: 0.9746
Epoch 3/20
13167/13167 [==============================] - 958s 73ms/step - loss: 0.9391 - accuracy: 0.8123
Epoch 4/20
13167/13167 [==============================] - 979s 74ms/step - loss: 0.9109 - accuracy: 0.8105
Epoch 5/20
13167/13167 [==============================] - 993s 75ms/step - loss: 0.9182 - accuracy: 0.8113
Epoch 6/20
13167/13167 [==============================] - 996s 76ms/step - loss: 0.9217 - accuracy: 0.8097
Epoch 7/20
13167/13167 [==============================] - 999s 76ms/step - loss: 0.9138 - accuracy: 0.8123
Epoch 8/20
13167/13167 [==============================] - 1001s 76ms/step - loss: 0.9176 - accuracy: 0.8093
Epoch 9/20
13167/13167 [==============================] - 1008s 77ms/step - loss: 0.9154 - accuracy: 0.8096
Epoch 10/20
13167/13167 [======

In [46]:
with open('historySeq.json', 'w') as f:
    json.dump(historySeq.history, f)

In [55]:
with open('historyFF.json', 'w') as f:
    json.dump(historyFF.history, f)

In [19]:
with open('historyRNN.json', 'w') as f:
    json.dump(historyRNN.history, f)

In [27]:
with open('historyCNN.json', 'w') as f:
    json.dump(historyCNN.history, f)

In [16]:
with open('historyCNNsep.json', 'w') as f:
    json.dump(historyCNNsep.history, f)